### Project Proposal - Heart Disease dataset

#### Introduction

Cardiovascular disease (also known as heart disease) is a type of disease that affects the heart or blood vessels. Patients suffering from this disease have compromised circulatory systems due to several factors including age, sex, cholestrol levels etc. 

In this project, our goal is to identify which chest pain type is more likely to indicate heart disease. For this classification, we have chosen to use the Heart Disease dataset from the UCI Machine Learning Repository (UCI). The dataset contains many variables which affect the probability of a chest pain type being indicative of heart disease. For our data analysis, we will be focusing on the following variables:

- Age - age in years
- Sex - (1=male, 0=female)
- Chest pain type (cp) - 1 = typical angina, 2 = atypical angina, 3 = non-angina pain, 4 = asymptomatic angina
- Fasting blood sugar (fbs) - fasting blood sugar in mg/dL 
- Maximum heart rate achieved - measured in beats per min (BPM)
- Exercise induced angina - 1 = yes; 0 = no
- Diagnosis - diagnosis of angiographic (heart) disease

In [6]:
# Packages needed for classification on our dataset
import altair as alt
import numpy as np
import pandas as pd
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

alt.data_transformers.disable_max_rows()
alt.renderers.enable("mimetype")

RendererRegistry.enable('mimetype')

### Data analysis

In [7]:
# Loading the dataset using the URL
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

heart_disease = pd.read_csv(url, header=None)

heart_disease.columns = ("Age",
           "Sex",
           "Chest_Pain_Type",
           "Resting_Blood_Pressure",
           "Serum_Cholesterol",
           "Fasting_Blood_Sugar",
           "Resting_ECG",
           "Max_Heart_Rate",
           "Exercise_Induced_Angina",
           "ST_Depression_Exercise",
           "Peak_Exercise_ST_Segment",
           "Num_Major_Vessels_Flouro",
           "Thalassemia",
           "Diagnosis") # assigning human-readable column headings

# Filtering to include only the columns we will be using in our analysis
heart_disease_final = heart_disease.loc[:, ["Age","Sex","Chest_Pain_Type","Fasting_Blood_Sugar","Max_Heart_Rate","Exercise_Induced_Angina","Diagnosis"]]
heart_disease_final


,Age,Sex,Chest_Pain_Type,Fasting_Blood_Sugar,Max_Heart_Rate,Exercise_Induced_Angina,Diagnosis
0,63.0,1.0,1.0,1.0,150.0,0.0,0
1,67.0,1.0,4.0,0.0,108.0,1.0,2
2,67.0,1.0,4.0,0.0,129.0,1.0,1
3,37.0,1.0,3.0,0.0,187.0,0.0,0
4,41.0,0.0,2.0,0.0,172.0,0.0,0
...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,0.0,132.0,0.0,1
299,68.0,1.0,4.0,1.0,141.0,0.0,2
300,57.0,1.0,4.0,0.0,115.0,1.0,3
301,57.0,0.0,2.0,0.0,174.0,0.0,1


Since our data was already in a tidy format (each column is a single variable, each row is a single observation and each cell is a single value). Hence, the only steps we needed to take to make the dataset easier to work with were to drop all rows with Na's and change the data types of some columns. This is shown below:

In [8]:
heart_disease_final.dropna() # Removing Na's


# Age is measured in years therefore will be an integer
heart_disease_final.Age = heart_disease.Age.astype('int64') 

# Since sex is one of male or female, we convert the data type to bool
heart_disease_final.Sex = heart_disease.Sex.astype(bool) 

# A patient either has heart disease or not, hence diagnosis will also be of bool type
heart_disease_final.Diagnosis = heart_disease.Diagnosis.astype(bool) 

# Since chest pain type can only be one of 1, 2, 3, 4 hence it will be of type int64
heart_disease_final.Chest_Pain_Type = heart_disease.Chest_Pain_Type.astype('int64') 

heart_disease_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      303 non-null    int64  
 1   Sex                      303 non-null    bool   
 2   Chest_Pain_Type          303 non-null    int64  
 3   Fasting_Blood_Sugar      303 non-null    float64
 4   Max_Heart_Rate           303 non-null    float64
 5   Exercise_Induced_Angina  303 non-null    float64
 6   Diagnosis                303 non-null    bool   
dtypes: bool(2), float64(3), int64(2)
memory usage: 12.6 KB


In [9]:
heart_disease_final # The final tidy data

,Age,Sex,Chest_Pain_Type,Fasting_Blood_Sugar,Max_Heart_Rate,Exercise_Induced_Angina,Diagnosis
0,63,True,1,1.0,150.0,0.0,False
1,67,True,4,0.0,108.0,1.0,True
2,67,True,4,0.0,129.0,1.0,True
3,37,True,3,0.0,187.0,0.0,False
4,41,False,2,0.0,172.0,0.0,False
...,...,...,...,...,...,...,...
298,45,True,1,0.0,132.0,0.0,True
299,68,True,4,1.0,141.0,0.0,True
300,57,True,4,0.0,115.0,1.0,True
301,57,False,2,0.0,174.0,0.0,True


### Splitting data into training and testing data

In [14]:
heart_train, heart_test = train_test_split(heart_disease_final, test_size=0.25, random_state=123) # set the random state to be 123
print(heart_train.head())
print(heart_test.head())

     Age    Sex  Chest_Pain_Type  Fasting_Blood_Sugar  Max_Heart_Rate  \
36    43   True                4                  0.0           120.0   
148   45   True                2                  0.0           170.0   
21    58  False                1                  1.0           162.0   
187   66   True                2                  0.0           120.0   
161   77   True                4                  0.0           162.0   

     Exercise_Induced_Angina  Diagnosis  
36                       1.0       True  
148                      0.0      False  
21                       0.0      False  
187                      1.0       True  
161                      1.0       True  
     Age    Sex  Chest_Pain_Type  Fasting_Blood_Sugar  Max_Heart_Rate  \
11    56  False                2                  0.0           153.0   
292   44   True                4                  0.0           144.0   
269   42   True                3                  0.0           150.0   
268   40   True  

### Summary Tables

First, we started by creating a dataframe grouped by chest pain type to see summarized count of each variable for each chest pain type

In [17]:
# Grouping by Chest_Pain_Type to see count of each variable for every chest pain type
chest_pain_grouped = heart_train.groupby("Chest_Pain_Type").agg("count")
chest_pain_grouped

,Age,Sex,Fasting_Blood_Sugar,Max_Heart_Rate,Exercise_Induced_Angina,Diagnosis
Chest_Pain_Type,,,,,,
1,16,16,16,16,16,16
2,35,35,35,35,35,35
3,72,72,72,72,72,72
4,104,104,104,104,104,104


## We created a table in order to count the amount of chest pain type experienced in general.

In [21]:
# Selecting the needed columns to utilize in counting the amount of chest pain type experienced
chest_pain_type_count = heart_train.loc[:,["Chest_Pain_Type","Age"]]

# Grouping by chest pain type and counting how many experience each chest pain type
chest_pain_type_count = chest_pain_type_count.groupby("Chest_Pain_Type").agg(["count"]).reset_index()

# Renaming the columns into the correct names
chest_pain_type_count.columns=("Chest_Pain_Type", "Count")
chest_pain_type_count

,Chest_Pain_Type,Count
0,1,16
1,2,35
2,3,72
3,4,104


#### Count of True Diagnosis for each Chest Pain Type

We created a table that shows the chest pain type and the count of positive diagnosis (meaning the person experiencing that chest pain type was diagnosed with heart disease). 

In [19]:
# Taking only the rows where diagnosis is 1
diagnosis_true = heart_train[heart_train["Diagnosis"]==True] 
diagnosis_true

# Grouping by chest pain type and counting the variables for each class
chest_pain = diagnosis_true.groupby("Chest_Pain_Type").agg("count").reset_index()
chest_pain

# Showing only the columns Chest_Pain_Type and Diagnosis
chest_pain_true = chest_pain.loc[:, ["Chest_Pain_Type", "Diagnosis"]]
chest_pain_true.columns = ("Chest_Pain_Type", "True_Diagnosis_Count")
chest_pain_true

,Chest_Pain_Type,True_Diagnosis_Count
0,1,5
1,2,6
2,3,16
3,4,78


#### Count of False Diagnosis for each Chest Pain Type

In [23]:
# Taking only the rows where diagnosis is 1
diagnosis_false = heart_train[heart_train["Diagnosis"]==False] 
diagnosis_false

# # Grouping by chest pain type and counting the variables for each class
chest_pain = diagnosis_false.groupby("Chest_Pain_Type").agg("count").reset_index()
chest_pain

# # Showing only the columns Chest_Pain_Type and Diagnosis
chest_pain_false = chest_pain.loc[:, ["Chest_Pain_Type", "Diagnosis"]]
chest_pain_false.columns = ("Chest_Pain_Type", "False_Diagnosis_Count")
chest_pain_false

,Chest_Pain_Type,False_Diagnosis_Count
0,1,11
1,2,29
2,3,56
3,4,26


### Visualizations